In [49]:
import xml.etree.ElementTree as ET
import xmljson
import pandas as pd
import pyodbc
import os
from pprint import pprint
import xmltodict

pd.set_option('max_colwidth', 160)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"\
    r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"

def vtb_broker_security_deals(path):
    """Считываем данные по движению денежных средств из брокерского отчёта Банка ВТБ в формате xml"""
    
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0   
    
    # Считываем данные из файлов
    for file in files:
        tree = ET.parse(os.path.join(directory, path, file))    
        root = tree.getroot()
        json = xmljson.gdata.data(root[17][0])
        json = json['{report577p_v1}Подробности9_Collection']['{report577p_v1}Подробности9']
        data = pd.DataFrame(json)
        if counter == 0:
            security_deals = data
            counter += 1
        else:
            security_deals = transactions.append(data, ignore_index=True) 
                
    # Обработка данных
    security_deals = security_deals[['NameBeg9','curs_datebeg9','currency_ISO9',
                                     'NameEnd9','currency_paym7','deal_cost7',
                                     'bank_сommition7','deal_code5']]
    security_deals.columns = ['security','date', 'deal_type', 'quantity',
                              'amount', 'accumulated coupon', 'deal_сommition', 'bank_сommition']
    security_deals.date = pd.to_datetime(security_deals.date, format='%Y-%m-%d')
#    security_deals.loc[:, 'date'] = pd.to_datetime(transactions.date, format='%Y-%m-%d')
#     transactions.description = transactions.description + '; ' + transactions.notes
#     transactions = transactions[['date', 'amount', 'description']]
#     transactions = transactions.sort_values(by=['date'])

    return security_deals

path = r'Георгий/Брокерские отчёты XML'
security_deals = vtb_broker_security_deals(path)
security_deals

,security,date,deal_type,quantity,amount,accumulated coupon,deal_сommition,bank_сommition
0,"Газпром а.о., 1-02-00028-A, RU0007661625",2020-02-26 13:15:54,Покупка,80.0,17824.80,0.00,1.78,6.73
1,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-26 13:16:44,Покупка,5.0,8775.00,0.00,0.87,3.31
2,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-26 13:21:27,Покупка,6.0,10548.00,0.00,1.06,3.98
3,"Газпром а.о., 1-02-00028-A, RU0007661625",2020-02-27 09:59:40,Покупка,80.0,17679.20,0.00,1.77,6.68
4,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-27 09:59:44,Покупка,5.0,8752.50,0.00,0.87,3.30
...,...,...,...,...,...,...,...,...
345,"ЛомМасБОП7, 4B02-07-36516-R-001P, RU000A101FT1",2020-07-27 12:26:53,Покупка,30.0,26597.40,782.40,2.58,9.75
346,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-28 18:04:34,Покупка,26.0,19266.00,0.00,1.93,7.27
347,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-29 11:02:20,Покупка,25.0,18510.00,0.00,1.85,6.99
348,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-30 15:13:19,Покупка,10.0,7360.00,0.00,0.73,2.78


In [35]:
df = security_deals
df

,NameBeg9,curs_datebeg9,currency_ISO9,NameEnd9,deal_count7,deal_price7,currency_price7,currency_paym7,deal_cost7,bank_сommition7,deal_code5,deliv_date6,paym_date5,request_number,deal_code1,uds_befor,contr_name5,deal_place7,notes7
0,"Газпром а.о., 1-02-00028-A, RU0007661625",2020-02-26T13:15:54,Покупка,80.0,RUR,222.81,RUR,17824.80,0.00,1.78,6.73,2020-02-28T09:00:00,2020-02-28T09:00:00,19334940200,B3103078226,1.31,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
1,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-26T13:16:44,Покупка,5.0,RUR,1755.00,RUR,8775.00,0.00,0.87,3.31,2020-02-28T09:00:00,2020-02-28T09:00:00,19334957663,B3103079669,1.31,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
2,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-26T13:21:27,Покупка,6.0,RUR,1758.00,RUR,10548.00,0.00,1.06,3.98,2020-02-28T09:00:00,2020-02-28T09:00:00,19335088330,B3103089622,1.32,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
3,"Газпром а.о., 1-02-00028-A, RU0007661625",2020-02-27T09:59:40,Покупка,80.0,RUR,220.99,RUR,17679.20,0.00,1.77,6.68,2020-03-02T09:00:00,2020-03-02T09:00:00,19341592578,B3103796348,1.38,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
4,"Башнефть а.п., 2-01-00013-A, RU0007976965",2020-02-27T09:59:44,Покупка,5.0,RUR,1750.50,RUR,8752.50,0.00,0.87,3.30,2020-03-02T09:00:00,2020-03-02T09:00:00,19341603715,B3103796983,1.38,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,"ЛомМасБОП7, 4B02-07-36516-R-001P, RU000A101FT1",2020-07-27T12:26:53,Покупка,30.0,RUR,86.05,RUR,26597.40,782.40,2.58,9.75,2020-07-28T09:00:00,2020-07-28T09:00:00,20829434419,B3245547587,2.61,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
346,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-28T18:04:34,Покупка,26.0,RUR,741.00,RUR,19266.00,0.00,1.93,7.27,2020-07-30T09:00:00,2020-07-30T09:00:00,20844475900,B3247642677,2.65,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
347,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-29T11:02:20,Покупка,25.0,RUR,740.40,RUR,18510.00,0.00,1.85,6.99,2020-07-31T09:00:00,2020-07-31T09:00:00,20847676338,B3248053926,2.74,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
348,"ЛСР ао, 1-01-55234-E, RU000A0JPFP0",2020-07-30T15:13:19,Покупка,10.0,RUR,736.00,RUR,7360.00,0.00,0.73,2.78,2020-08-03T09:00:00,2020-08-03T09:00:00,20861056005,B3249817538,2.71,НКО НКЦ (АО),Фондовый рынок ПАО Московская Биржа,NaN
